### Note: Model is trained on Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import unicodedata
import string
import re
import random
import time
import datetime
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
import torchtext

import spacy
import numpy as np
import pandas as pd

## Read data

In [3]:
data_path = "/content/drive/MyDrive/g2p/data_new/rum/"
root_path = "/content/drive/MyDrive/g2p/"

In [4]:
df_rum_train = pd.read_csv(data_path + "rum_train.tsv", sep='\t', header=None, encoding='utf-8').rename(columns={0:"grapheme", 1:"phoneme"})
df_rum_dev = pd.read_csv(data_path + "rum_dev.tsv", sep='\t', header=None, encoding='utf-8').rename(columns={0:"grapheme", 1:"phoneme"})


In [5]:
df_rum_train.head()

,grapheme,phoneme
0,laude,l a u d e
1,foarte,f o̯ a r t e
2,scoli,s k o lʲ
3,circul,t͡ʃ i r k u l
4,buzunar,b u z u n a r


In [6]:
df_rum_dev.head()

,grapheme,phoneme
0,conduci,k o n d u t͡ʃʲ
1,frumoase,f r u m o̯ a s e
2,palma,p a l m a
3,caise,k a i s e
4,informatica,i n f o r m a t i k a


In [7]:
df_rum_train.shape

(700, 2)

In [8]:
df_rum_dev.shape

(100, 2)

## Use torchtext to prepare data

In [9]:
def tokenize_grapheme(text):
    """
    Tokenizes French text from a string into a list of strings (tokens)
    """
    return list(text)

In [10]:
def tokenize_phoneme(text):
    """
    Tokenizes French text from a string into a list of strings (tokens)
    """
    return text.split()

In [11]:
df_rum_train_gra = df_rum_train["grapheme"].apply(tokenize_grapheme)

df_rum_train_gra

0                  [l, a, u, d, e]
1               [f, o, a, r, t, e]
2                  [s, c, o, l, i]
3               [c, i, r, c, u, l]
4            [b, u, z, u, n, a, r]
                  ...             
695       [d, e, l, i, c, i, o, s]
696                [f, o, r, m, a]
697                [p, r, i, m, e]
698          [r, e, z, u, l, t, e]
699    [i, n, t, e, r, v, i, n, e]
Name: grapheme, Length: 700, dtype: object

In [12]:
df_rum_train_pho = df_rum_train["phoneme"].apply(tokenize_phoneme)

df_rum_train_pho

0                  [l, a, u, d, e]
1              [f, o̯, a, r, t, e]
2                    [s, k, o, lʲ]
3             [t͡ʃ, i, r, k, u, l]
4            [b, u, z, u, n, a, r]
                  ...             
695     [d, e, l, i, t͡ʃ, j, o, s]
696                [f, o, r, m, a]
697                [p, r, i, m, e]
698          [r, e, z, u, l, t, e]
699    [i, n, t, e, r, v, i, n, e]
Name: phoneme, Length: 700, dtype: object

In [13]:
SRC = torchtext.legacy.data.Field(tokenize = tokenize_grapheme, 
            init_token = '<sos>', # since initial encoder hidden state is always set to zero, the network can figure out that the time step is 0 and this token is optional
            eos_token = '<eow>', 
            lower = True)
TRG = torchtext.legacy.data.Field(tokenize = tokenize_phoneme, 
            init_token = '<sow>', 
            eos_token = '<eow>', 
            lower = True)

In [14]:
assert SRC.tokenize("hello") == ['h', 'e', 'l', 'l', 'o']
assert TRG.tokenize("b a b ɛ ɬ") == ['b', 'a', 'b', 'ɛ', 'ɬ']

In [15]:
train, val = torchtext.legacy.data.TabularDataset.splits(
    path=data_path, train='rum_train.tsv',validation='rum_dev.tsv', 
    format='tsv', skip_header=False, fields=[('SRC', SRC), ('TRG', TRG)])

test = torchtext.legacy.data.TabularDataset(
    path=data_path + 'new_rum_test.tsv',
    format='tsv', skip_header=False, fields=[('SRC', SRC), ('TRG', TRG)])

In [16]:
print(next(iter(train)).TRG)

['l', 'a', 'u', 'd', 'e']


In [17]:
print(next(iter(train)).SRC)

['l', 'a', 'u', 'd', 'e']


In [18]:
# build vocab
SRC.build_vocab(train)
TRG.build_vocab(train)

In [19]:
len(SRC.vocab)

30

In [20]:
print(SRC.vocab.itos)

['<unk>', '<pad>', '<sos>', '<eow>', 'e', 'i', 'a', 'r', 't', 'c', 'n', 'o', 'u', 'l', 'm', 's', 'p', 'd', 'f', 'b', 'v', 'g', 'z', 'j', 'x', 'h', 'ă', 'ț', 'y', 'â']


In [21]:
print(TRG.vocab.itos)

['<unk>', '<pad>', '<sow>', '<eow>', 'a', 'e', 'i', 't', 'r', 'n', 'u', 'o', 's', 'l', 'k', 'm', 'p', 'd', 'f', 'b', 'j', 'v', 'ɡ', 'z', 't͡ʃ', 'e̯', 'ʒ', 'rʲ', 'o̯', 't͡ʃʲ', 'ʃ', 'w', 'd͡ʒ', 'iː', 'd͡ʒʲ', 'nʲ', 'zʲ', 'h', 'lʲ', 'pʲ', 'ŋ', 'ə', 'vʲ', 'kʲ', 'mʲ', 'sʲ', 't͡s', 'ɨ', 'ʃʲ']


In [22]:
print(TRG.vocab.stoi['<pad>'])

1


In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [24]:
from torchtext.legacy.data import Iterator, BucketIterator
train_iter = BucketIterator(
        train,
        batch_size=1,
        device=device,
        sort_key=len,
        shuffle=True)
dev_iter, test_iter = Iterator.splits(
        (val, test),
        batch_sizes=(1,1),
        device=device,
        sort=False,
        shuffle=False)

In [25]:
# batch example of training data
for batch in train_iter:
    src = batch.SRC
    trg = batch.TRG
    print('tensor size of source language:', src.shape)
    print('tensor size of target language:', trg.shape)
    print('the tensor of first example in target language:', trg[:,0])
    break

tensor size of source language: torch.Size([10, 1])
tensor size of target language: torch.Size([10, 1])
the tensor of first example in target language: tensor([ 2, 14, 11,  9, 24,  5, 16,  7,  5,  3], device='cuda:0')


## Build seq2seq+attention model

### Encoder

In [26]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, n_layers, dropout):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dropout = dropout
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, enc_hid_dim, n_layers, dropout=dropout, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.lstm(embedded)
       
        # outputs are always from the top hidden layer, if bidirectional outputs are concatenated.
        # outputs = [sequence_length, batch_size, hidden_dim * num_directions]
        # hidden = [num_layers * num_directions, batch_size, hidden_size]
        # cell = [num_layers * num_directions, batch_size, hidden_size]

        a, b, c = outputs.shape
        h_a, h_b, h_c = hidden.shape

        return outputs.view(a, b, 2, -1).mean(dim=2), (hidden.view(self.n_layers, 2, h_b, h_c).mean(dim=1).squeeze(0), cell.view(self.n_layers, 2, h_b, h_c).mean(dim=1))

### Attention

In [27]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.W_a = nn.Linear(enc_hid_dim + dec_hid_dim, dec_hid_dim)
        self.v_a = nn.Parameter(torch.rand(dec_hid_dim)) # same as doing nn.Linear(dec_hid_dim, 1, bias=False)
        self.neg_inf = torch.tensor(-1e7, device=device)
        
    def forward(self, hidden, encoder_outputs, attention_mask):
        #(decoder) hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim]
        #attention_mask = [batch_size, src_len]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len-1 times
        # print("attnetion forward encoder outputs: ", encoder_outputs.shape) # 2 x 2 x 512
        # print("attnetion forward hidden: ", hidden.shape) # 2 x 2 x 512
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim]
        
        # attention scoring function - part 1 - tanh(W_a[s;h])
        energy = torch.tanh(self.W_a(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]
        
        energy = energy.permute(0, 2, 1)
        
        #energy = [batch size, dec hid dim, src len]
        
        #v = [dec hid dim]
        
        v = self.v_a.repeat(batch_size, 1).unsqueeze(1)
        
        #v = [batch size, 1, dec hid dim]
        
        # attention scoring function - part 2 - v_a(tanh(W_a[s;h]))
        attention = torch.bmm(v, energy).squeeze(1)
        
        #attention= [batch size, src len]

        # before computing the softmax, set attention to pad tokens to -infinity
        attention[attention_mask] = self.neg_inf

        # attention scoring function - part 2 - softmax(v_a(tanh(W_a[s;h])))
        return F.softmax(attention, dim=1)

### Decoder

In [28]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, n_layers, dropout, attention):
        super().__init__()
        
        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(enc_hid_dim + emb_dim, dec_hid_dim, n_layers, dropout=dropout)
        
        self.fc_out = nn.Linear(enc_hid_dim + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell, encoder_outputs, attention_mask):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim]
        #attention_mask = [batch_size, src_len]
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        # get the attention probabilities
        attention_weights = self.attention(hidden, encoder_outputs, attention_mask)
                
        #attention_weights = [batch size, src len]
        
        attention_weights = attention_weights.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim]
        # perform weighted sum of encoder hidden states to get attention output
        weighted = torch.bmm(attention_weights, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim]
        # concatenate the attention outputs (or context vectors) with the current decoder input
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim) + emb dim]
        output, (hidden, cell) = self.rnn(rnn_input, (hidden.unsqueeze(0), cell))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        # classification over the entire word vocabulary
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0), cell, attention_weights

### Seq2seq with attention

In [29]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        # create encoder attention mask, set attention to pad tokens to -infinity 
        attention_mask = (src == SRC.vocab.stoi[SRC.pad_token]).transpose(0, 1)
        # attention_mask = [batch size, src len]
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        # save the encoder-decoder attention weights
        # all_attention_weights = [batch_size, trg len-1, src len ]
        all_attention_weights = torch.zeros(trg.shape[1], trg.shape[0]-1, src.shape[0])
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, (hidden, cell) = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden, cell, attention_weights = self.decoder(input, hidden, cell, encoder_outputs, attention_mask)
            
            # all_attention_weights[t-1] = [src len, batch size]
            all_attention_weights[:,t-1,:] = attention_weights.squeeze(1)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs,all_attention_weights

### Train the seq2seq model

In [30]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)

# hyperparameters
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
N_LAYERS = 1
LEARNING_RT = 0.001

# model components
attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT, attn)
model = Seq2Seq(enc, dec, device).to(device)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [31]:
# weight init ## pre-trained embeddings later
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(30, 256)
    (lstm): LSTM(256, 512, dropout=0.5, bidirectional=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (W_a): Linear(in_features=1024, out_features=512, bias=True)
    )
    (embedding): Embedding(49, 256)
    (rnn): LSTM(768, 512, dropout=0.5)
    (fc_out): Linear(in_features=1280, out_features=49, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [32]:
optimizer = optim.Adam(model.parameters())

In [33]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
print('<pad> token index: ',TRG_PAD_IDX)
## we will ignore the pad token in true target set
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

<pad> token index:  1


In [34]:
clip = 1
model.train()

for i, batch in enumerate(train_iter):

    src = batch.SRC
    trg = batch.TRG
    optimizer.zero_grad()
    output,_ = model(src, trg)
    #trg = [trg len, batch size]
    #output = [trg len, batch size, output dim]

    output_dim = output.shape[-1]

    output = output[1:].view(-1, output_dim)
    trg = trg[1:].view(-1)

    #trg = [(trg len - 1) * batch size]
    #output = [(trg len - 1) * batch size, output dim]

    loss = criterion(output, trg)

    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

    optimizer.step()

    print(loss/src.shape[1])
    break

    # 2, 2, 512; ==> 1, 2, 1024; <- 

tensor(3.8934, device='cuda:0', grad_fn=<DivBackward0>)


## Train

In [35]:
def train_func(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.SRC
        trg = batch.TRG
        
        optimizer.zero_grad()
        
        output,_ = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [36]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.SRC
            trg = batch.TRG

            output,_ = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [37]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [38]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train_func(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, dev_iter, criterion)
    
    end_time = time.time()
    state_dict_model = model.state_dict() 
    state = {
        'epoch': epoch,
        'state_dict': state_dict_model,
        'optimizer': optimizer.state_dict()
        }
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    torch.save(state, root_path + "models/rum/" +str(epoch+1)+".pt")
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f}')

Epoch: 01 | Time: 0m 11s
	Train Loss: 2.647
	 Val. Loss: 2.806
Epoch: 02 | Time: 0m 11s
	Train Loss: 2.201
	 Val. Loss: 1.672
Epoch: 03 | Time: 0m 11s
	Train Loss: 0.806
	 Val. Loss: 0.523
Epoch: 04 | Time: 0m 11s
	Train Loss: 0.347
	 Val. Loss: 0.346
Epoch: 05 | Time: 0m 11s
	Train Loss: 0.199
	 Val. Loss: 0.349
Epoch: 06 | Time: 0m 11s
	Train Loss: 0.159
	 Val. Loss: 0.371
Epoch: 07 | Time: 0m 11s
	Train Loss: 0.101
	 Val. Loss: 0.341
Epoch: 08 | Time: 0m 11s
	Train Loss: 0.091
	 Val. Loss: 0.390
Epoch: 09 | Time: 0m 11s
	Train Loss: 0.068
	 Val. Loss: 0.326
Epoch: 10 | Time: 0m 11s
	Train Loss: 0.046
	 Val. Loss: 0.357


## Evluation

In [39]:
def inference(model, dev_iter, src_vocab, trg_vocab, attention = True, max_trg_len = 64):
    '''
    Function for translation inference

    Input: 
    model: translation model;
    trg_vocab: Target torchtext Field
    attention: the model returns attention weights or not.
    max_trg_len: the maximal length of translation text (optinal), default = 64

    Output:
    Corpus BLEU score.
    '''
    from torchtext.legacy.data import TabularDataset
    from torchtext.legacy.data import Iterator

    # convert index to text string
    def convert_itos(convert_vocab, token_ids):
        list_string = []
        for i in token_ids:
            if i == convert_vocab.vocab.stoi['<eos>']:
                break
            else:
                token = convert_vocab.vocab.itos[i]
                list_string.append(token)
        return list_string
  
    model.eval()
    all_trg = []
    all_translated_trg = []

    TRG_PAD_IDX = trg_vocab.vocab.stoi[trg_vocab.pad_token]

    with torch.no_grad():
    
        for i, batch in enumerate(dev_iter):

            src = batch.SRC
            #src = [src len, batch size]

            trg = batch.TRG
            #trg = [trg len, batch size]

            batch_size = trg.shape[1]

            # create a placeholder for traget language with shape of [max_trg_len, batch_size] where all the elements are the index of <pad>. Then send to device
            trg_placeholder = torch.Tensor(max_trg_len, batch_size)
            trg_placeholder.fill_(TRG_PAD_IDX)
            trg_placeholder = trg_placeholder.long().to(device)
            if attention == True:
              output,_ = model(src, trg_placeholder, 0) #turn off teacher forcing
            else:
              output,_ = model(src, trg_placeholder, 0) #turn off teacher forcing
            # get translation results, we ignor first token <sos> in both translation and target sentences. 
            # output_translate = [(trg len - 1), batch, output dim] output dim is size of target vocabulary.
            output_translate = output[1:]
            # store gold target sentences to a list 
            all_trg.append(trg[1:].cpu())

            # Choose top 1 word from decoder's output, we get the probability and index of the word
            prob, token_id = output_translate.data.topk(1)
            translation_token_id = token_id.squeeze(2).cpu()

            # store gold target sentences to a list 
            all_translated_trg.append(translation_token_id)
      
    all_gold_text = []
    all_translated_text = []
    for i in range(len(all_trg)): 
        cur_gold = all_trg[i]
        cur_translation = all_translated_trg[i]
        for j in range(cur_gold.shape[1]):
            gold_convered_strings = convert_itos(trg_vocab,cur_gold[:,j])
            trans_convered_strings = convert_itos(trg_vocab,cur_translation[:,j])

            all_gold_text.append(gold_convered_strings)
            all_translated_text.append(trans_convered_strings)

    return all_gold_text, all_translated_text

In [40]:
def get_validation_score(model_best, dev_iter, src_vocab, trg_vocab, out_fp):
  gold, translated = inference(model_best, dev_iter, src_vocab, trg_vocab, True, 64)
  processed_gold = []
  for word in gold:
    curr_word = []
    for char in word:
      if char != "<eow>" and char != "<pad>":
        curr_word.append(char)
    processed_gold.append(curr_word)
  processed_translated = []
  for word in translated:
    curr_word = []
    for char in word:
      if char == "<eow>":
        break
      if char != "<eow>":
        curr_word.append(char)
    processed_translated.append(curr_word)
  with open(out_fp, "w", encoding="utf-8") as fout:
    for g, t in zip(processed_gold, processed_translated):
      fout.write(" ".join(g) + "\t" + " ".join(t) + "\n")

In [41]:
%cd /content/drive/MyDrive/g2p

/content/drive/MyDrive/g2p


In [42]:
for i in range(1, 11):
  print("epoch:", i)
  checkpoint = torch.load(root_path+f"models/rum/{i}.pt")
  model.load_state_dict(checkpoint['state_dict'])
  get_validation_score(model, dev_iter, SRC, TRG, root_path+"evaluation/rum_gold_translated.tsv")
  !python evaluate.py /content/drive/MyDrive/g2p/evaluation/rum_gold_translated.tsv

epoch: 1
WER:	100.00
PER:	76.90
epoch: 2
WER:	98.00
PER:	50.08
epoch: 3
WER:	36.00
PER:	8.77
epoch: 4
WER:	16.00
PER:	4.22
epoch: 5
WER:	17.00
PER:	4.38
epoch: 6
WER:	19.00
PER:	4.05
epoch: 7
WER:	17.00
PER:	3.71
epoch: 8
WER:	21.00
PER:	5.56
epoch: 9
WER:	12.00
PER:	2.70
epoch: 10
WER:	14.00
PER:	3.04


## Prediction

In [48]:
checkpoint = torch.load(root_path+f"models/rum/9.pt")
model.load_state_dict(checkpoint['state_dict'])
_, translated = inference(model, test_iter, SRC, TRG, True, 64)

In [49]:
def generate_final_prediction(test_file_path, translated, output_file_path):
  '''
  test_file_path -- the file path to the test file
  translated -- a list of lists that contain predictions
  output_file_path -- output file path that you will create
  '''
  with open(test_file_path, "r", encoding="utf-8") as f:
    gold = f.read().splitlines()
  processed_translated = []
  for word in translated:
    curr_word = []
    for char in word:
      if char == "<eow>":
        break
      if char != "<eow>" and char != "<pad>":
        curr_word.append(char)
    processed_translated.append(curr_word)
  with open(output_file_path, "w", encoding="utf-8") as fout:
    for g, t in zip(gold, processed_translated):
      fout.write(g + "\t" + " ".join(t) + "\n")

In [50]:
rum_test_path = data_path + "rum_test.tsv"

In [51]:
generate_final_prediction(rum_test_path, translated, root_path+"predictions/rum_preds.tsv")